<a href="https://colab.research.google.com/github/alexandergribenchenko/Test_R5_DE/blob/main/Solucion_R5_Data_Enginner_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ¿Como crear una prueba de concepto de un Datawarehouse en el contexto del retail?
**Data Enginner Challenge - Grupo R5  (Presentado por: Alexander Ortega, Julio 2023)**

Este notebook presenta la solución al reto propuesto por el Grupo R5 para optar al rol de Data Engineer. El reto consiste en la implementación de un datawarehouse en el contexto de negocio del retail.

El notebook está compuesto de 3 secciones que dan respuesta a cada uno de los ítems descritos en la consigna compartida:

- **Sección 01. Diseño del datawarehouse:** Se presenta el diagrama del DWH implementado y se justifica la metodología implementada para su desarrollo.
- **Sección 02. Pipeline para la creación del Datawarehouse:** Se crea un pipeline en python que permite en 3 etapas implementar el DWH diseñado.
- **Sección 03. Respuesta a preguntas particulares de negocio:** A partir del DWH implementado se responden a 2 preguntas específicas de negocio.

**Nota para la ejecución:** Este notebook se desarrolló de manera plena `google colab` y lo único que se debe hacer para tener una ejecución completa es hacer una copia del mismo. Los orígenes y destinos de los datos están configurados de tal manera que no se debe hacer ninguna modificación adicional. Las 3 librerías empleadas están instaladas por defecto en los cluster que provee google colab (`pandas, sqlite3` y `os`) por lo cual no hay necesidad de instalaciones complementarias.

# Sección 01. Diseño del Datawarehouse

## S.01. Requerimiento

Diseñar un DWH con base en el dataset, se puede usar cualquier metodología.    
- Entregable: Gráfico representando el diseño, explicación de por qué se eligió la metodología teniendo en cuenta la escalabilidad y facilidad de uso para usuarios finales.

## S.01. Solución

### S.01.01. Gráfico con el diseño del Datawarehouse

![Texto alternativo](https://raw.githubusercontent.com/alexandergribenchenko/Test_R5_DE/main/images/DWH_R5_DE_Test.png)


### S.01.02. Justificación metodología

El diseño del Datawarehouse se generó bajo la metodología conocida como enfoque estrella (star schema).

En el enfoque estrella, se utiliza una tabla de hechos central que contiene las métricas o medidas clave que se desean analizar, como por ejemplo el total de venta de las órdenes. Esta tabla de hechos se conecta directamente con las tablas dimensionales a través de claves foráneas. Cada tabla dimensional representa una dimensión clave en el contexto de los datos, como el cliente, la tienda y la fecha.

En relación a la escalabilidad y a la facilidad del uso para usuarios finales podemos destacar los siguientes aspectos:


#### **Escalabilidad**

- **Agregación eficiente:** El enfoque estrella permite realizar agregaciones precalculadas y simplifica el cálculo de medidas en consultas analíticas. Esto mejora el rendimiento de las consultas y permite escalar el Data Warehouse para manejar grandes volúmenes de datos y consultas complejas de manera eficiente.
- **Incorporación de nuevas dimensiones:** El diseño del enfoque estrella facilita la incorporación de nuevas dimensiones en el futuro sin afectar la estructura existente. Esto brinda flexibilidad y escalabilidad para adaptarse a los cambios en los requisitos de análisis y las necesidades empresariales a medida que evolucionan con el tiempo.

#### **Facilidad de uso para usuarios finales**

- **Modelado intuitivo:** El enfoque estrella utiliza una estructura simple y fácil de comprender, lo que facilita a los usuarios finales comprender la relación entre las dimensiones y las medidas. Esto permite a los usuarios explorar y analizar los datos de manera más intuitiva, sin requerir un conocimiento profundo del modelo subyacente.
- **Consultas sencillas:** Las consultas en el enfoque estrella suelen ser más sencillas y directas, ya que se centran en las dimensiones clave y las medidas de interés. Los usuarios pueden formular consultas basadas en los atributos clave, como cliente, tienda o fecha, lo que facilita el análisis multidimensional y la obtención de información relevante rápidamente.

# Sección 02. Pipeline para la creación del Datawarehouse.

## S.02. Requerimiento

Crear un pipeline en Python que tome como input el dataset y que escriba el resultado en las tablas diseñadas en el punto 1. Se puede usar cualquier motor de base de datos.
- Entregable: Código en Python del pipeline, se puede usar cualquier librería que considere necesaria.

## S.02. Solución

Se desarrolló un pipeline en python con 3 etapas básicas. Cada una de las etapas emplea como base una función customizada que se define en el propio script:
- **Etapa 1 - Creación de la base de datos:** En esta etapa se crea una base de datos denominada `retail_database.db` y se genera en su interior las tablas con las estructuras y tipos de variables definidas en el diagrama propuesto para el DWH. La librería empleada para ello es sqlite3.
- **Etapa 2 - Obtención de los csv files para nutrir las tablas de facts y dimentions:** En esta etapa se generan las diferentes tablas de hechos y dimensiones en archivos `csv` a partir del archivo fuente compartido. Estas se almacenan en un folder denominado `data` en la raíz del directorio en que se encuentra el notebook.
- **Etapa 03. Carga de las tablas csv al DWH:** En esta etapa agrega la información de los archivos generados en la etapa 2, a las tablas que hacen parte del datawarehouse creado en la etapa 1.

In [1]:
import pandas as pd
import sqlite3
import os

def create_database_structure(database_name):

  conn = sqlite3.connect(database_name)
  c = conn.cursor()

  dict_query = {}

  dict_query['query_DimBarrio'] = """
  CREATE TABLE IF NOT EXISTS "DimBarrio" (
    "id_barrio" int,
    "nombre_barrio" varchar,
    PRIMARY KEY ("id_barrio")
  )
  """

  dict_query['query_DimTienda'] = """
  CREATE TABLE IF NOT EXISTS "DimTienda" (
    "id_tienda" int,
    "tipo_tienda" varchar,
    "latitud_tienda" float,
    "longitud_tienda" float,
    "id_barrio" int,
    PRIMARY KEY ("id_tienda"),
    CONSTRAINT "FK_DimTienda.id_barrio"
      FOREIGN KEY ("id_barrio")
        REFERENCES "DimBarrio"("id_barrio")
  )
  """

  dict_query['query_DimCliente'] = """
  CREATE TABLE IF NOT EXISTS "DimCliente" (
    "id_cliente" int,
    "tipo_documento" int,
    PRIMARY KEY ("id_cliente")
  )
  """

  dict_query['query_DimFecha'] = """
  CREATE TABLE IF NOT EXISTS "DimFecha" (
    "fecha_compra" datetime,
    PRIMARY KEY ("fecha_compra")
  )
  """

  dict_query['query_FactOrden'] = """
  CREATE TABLE IF NOT EXISTS "FactOrden" (
    "id_orden" int,
    "total_compra" float,
    "fecha_compra" datetime,
    "id_cliente" int,
    "id_tienda" int,
    PRIMARY KEY ("id_orden"),
    CONSTRAINT "FK_FactOrden.fecha_compra"
      FOREIGN KEY ("fecha_compra")
        REFERENCES "DimFecha"("fecha_compra"),
    CONSTRAINT "FK_FactOrden.id_tienda"
      FOREIGN KEY ("id_tienda")
        REFERENCES "DimTienda"("id_tienda"),
    CONSTRAINT "FK_FactOrden.id_cliente"
      FOREIGN KEY ("id_cliente")
        REFERENCES "DimCliente"("id_cliente")
  )
  """

  for query in dict_query.keys():
    c.execute(dict_query[query])

  conn.commit()
  conn.close()

  return print('Creación de la base de datos finalizada con exito')

def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    return folder_name

def create_csvs_fact_dim(path_input, path_output):

  df = pd.read_csv(path_input, dtype=object)

  df_DimBarrio  =  df[['id_barrio', 'nombre_barrio']].drop_duplicates().\
                  sort_values(by='id_barrio', ascending=True).\
                  reset_index(drop=True)

  df_DimTienda  =  df[['codigo_tienda', 'tipo_tienda', 'latitud_tienda', 'longitud_tienda', 'id_barrio']].drop_duplicates().\
                  rename(columns={'codigo_tienda': 'id_tienda'}).\
                  sort_values(by='id_tienda', ascending=True).\
                  reset_index(drop=True)

  df_DimCliente  =  df[['num_documento_cliente', 'tipo_documento_cliente']].drop_duplicates().\
                    sort_values(by='num_documento_cliente', ascending=True).\
                    rename(columns={'num_documento_cliente': 'id_cliente'}).\
                    reset_index(drop=True)

  df_DimFecha  =  df[['fecha_compra']].drop_duplicates().\
                  sort_values(by='fecha_compra', ascending=True).\
                  reset_index(drop=True)

  df_FactOrden  =   df.reset_index()[['index','total_compra', 'fecha_compra', 'num_documento_cliente', 'codigo_tienda']].drop_duplicates().\
                  rename(columns={'index': 'id_compra', 'codigo_tienda': 'id_tienda', 'num_documento_cliente': 'id_cliente'}).\
                  sort_values(by='id_compra', ascending=True).\
                  reset_index(drop=True)

  table_names = ['DimBarrio', 'DimTienda',  'DimCliente', 'DimFecha', 'FactOrden']

  for table_name in table_names:
      codigo_download_csv = f"df_{table_name}.to_csv(f'{create_folder(path_output)}/{table_name}.csv', index=False)"
      exec(codigo_download_csv)

  return print('Archivos csv almacenados con exito')

def populate_db_tables(database_name, path):

    conn = sqlite3.connect(database_name)
    c = conn.cursor()

    table_names = ['DimBarrio', 'DimTienda',  'DimCliente', 'DimFecha', 'FactOrden']

    for table_name in table_names:
      codigo_carga_df_from_csv = f"df_{table_name} = pd.read_csv(path + '{table_name}.csv')"
      codigo_carga_tabla_from_df = f"df_{table_name}.to_sql('{table_name}', conn, if_exists='replace', index=False)"
      exec(codigo_carga_df_from_csv)
      exec(codigo_carga_tabla_from_df)

    conn.close()

    return print('La carga de las tablas se realizó exitosamante')

if __name__ == '__main__':

  print('----------- PIPELINE -----------')
  path_input = 'https://raw.githubusercontent.com/alexandergribenchenko/Test_R5_DE/main/data/dataset.csv'
  path_output = './data'
  path ='./data/'
  database_name = 'retail_database.db'

  try:
    print('--->>> Etapa 01. Creación de la base de datos')
    create_database_structure(database_name)
  except:
    print('Fallo en la Etapa 01')

  try:
    print('--->>> Etapa 02. Obtención de los csv files para nutrir las tablas de facts y dimentions')
    create_csvs_fact_dim(path_input, path_output)
  except:
    print('Fallo en la Etapa 02')

  try:
    print('--->>> Etapa 03. Carga de las tablas csv al DWH')
    populate_db_tables(database_name, path)
  except:
    print('Fallo en la Etapa 03')

----------- PIPELINE -----------
--->>> Etapa 01. Creación de la base de datos
Creación de la base de datos finalizada con exito
--->>> Etapa 02. Obtención de los csv files para nutrir las tablas de facts y dimentions
Archivos csv almacenados con exito
--->>> Etapa 03. Carga de las tablas csv al DWH
La carga de las tablas se realizó exitosamante


# Sección 03. Respuesta a preguntas particulares de negocio.

## S.03. Requerimiento

Una vez los datos estén cargados a la base de datos utilice Python y SQL para responder las siguientes preguntas:
1. ¿Cuáles son las tiendas con compras de al menos 100 clientes diferentes?
2. ¿Cuáles son los 5 barrios donde la mayor cantidad de clientes únicos realizan compras en tiendas tipo 'Tienda Regional'?
- Entregable: Código en Python y SQL resolviendo cada una de las preguntas.

## S.03. Solución

Una vez creado nuestro DWH, hacemos la conexión al mismo para responder a cada una de las preguntas planteadas. Las respuesta a cada una de las preguntas se presentan de 2 maneras diferentes: la primera haciendo uso del lenguaje de SQL y la segunda empleando python. Sin embargo, como es de esperarse, se evidencia la coincidencia en las salidas por cualquiera de los 2 caminos.

In [2]:
# Conectarse a la base de datos SQLite
conn = sqlite3.connect('retail_database.db')

### S.03.01. Tiendas con compras de al menos 100 clientes diferentes.

En el presente numeral se entrega un listado del id de las tiendas que tienen cuanto menos 100 clientes únicos. Además, para efectos de claridad, se incluye una columna adicional que presenta la cantidad de clientes únicos para cada tienda y se ordena de manera decreciente respecto a esta columna.

#### Versión SQL

In [3]:
pd.read_sql_query("""
SELECT id_tienda, COUNT(DISTINCT id_cliente) AS cantidad_clientes_unicos
FROM FactOrden
GROUP BY id_tienda
HAVING COUNT(DISTINCT id_cliente) >= 100
ORDER BY cantidad_clientes_unicos DESC;
""", conn, dtype=object)

,id_tienda,cantidad_clientes_unicos
0,745,660
1,11303609,525
2,829,482
3,812,457
4,747,457
...,...,...
248,2037,102
249,1187,102
250,2182,101
251,4594,100


#### Versión Python

In [4]:
df_FactOrden = pd.read_sql_query("SELECT * FROM FactOrden", conn, dtype=object)
df_Q_03_01 = df_FactOrden.groupby(['id_tienda'], as_index=False).\
           agg(cantidad_clientes_unicos=('id_cliente', lambda x: len(set(x)))).\
           sort_values(by=['cantidad_clientes_unicos'], ascending=[False]).reset_index(drop=True)
df_Q_03_01 = df_Q_03_01[df_Q_03_01.cantidad_clientes_unicos>=100]
df_Q_03_01

,id_tienda,cantidad_clientes_unicos
0,745,660
1,11303609,525
2,829,482
3,812,457
4,747,457
...,...,...
248,1187,102
249,2037,102
250,2182,101
251,3588,100


### S.03.02. Top 5 de los barrios donde la mayor cantidad de clientes únicos realizan compras en tiendas tipo 'Tienda Regional'.

En el presente numeral se entrega un listado del id y nombre de los barrios que presentan la mayor cantidad de clientes únicos que realizan compras en tiendas de tipo Tienda Regional'. Además, para efectos de claridad, se incluye una columna adicional que presenta la cantidad de clientes únicos para cada barrio y se ordena de manera decreciente respecto a esta columna.

#### Versión SQL

In [5]:
pd.read_sql_query("""
WITH TiendasTipoRegional AS (
    SELECT id_tienda, tipo_tienda, id_barrio
    FROM DimTienda
    WHERE tipo_tienda = 'Tienda Regional'
)
SELECT dimb.id_barrio, dimb.nombre_barrio, COUNT(DISTINCT fo.id_cliente) AS cantidad_clientes_unicos
FROM FactOrden fo
JOIN TiendasTipoRegional ttr ON fo.id_tienda=ttr.id_tienda
JOIN DimBarrio dimb ON ttr.id_barrio= dimb.id_barrio
GROUP BY dimb.id_barrio
ORDER BY cantidad_clientes_unicos DESC
LIMIT 5
""", conn, dtype=object)

,id_barrio,nombre_barrio,cantidad_clientes_unicos
0,738000377,Unicentro Cali,354
1,737998204,Versalles,328
2,737999419,El Sena,287
3,737998095,Unidad Residencial Santiago de Cali,276
4,737998838,Calima,250


#### Versión Python

In [6]:
df_FactOrden = pd.read_sql_query("SELECT * FROM FactOrden", conn, dtype=object)
df_DimTienda = pd.read_sql_query("SELECT * FROM DimTienda", conn, dtype=object)
df_DimBarrio = pd.read_sql_query("SELECT * FROM DimBarrio", conn, dtype=object)

df_DimTienda_Filtered = df_DimTienda[df_DimTienda.tipo_tienda=='Tienda Regional']

df_join = pd.merge(df_FactOrden, df_DimTienda_Filtered, on='id_tienda', how='right')
df_join = pd.merge(df_join, df_DimBarrio, on='id_barrio', how='left')

df_Q_03_02 = df_join.groupby(['id_barrio', 'nombre_barrio'], as_index=False).\
           agg(cantidad_clientes_unicos=('id_cliente', lambda x: len(set(x)))).\
           sort_values(by=['cantidad_clientes_unicos'], ascending=[False]).reset_index(drop=True)
df_Q_03_02 = df_Q_03_02.head()
df_Q_03_02

,id_barrio,nombre_barrio,cantidad_clientes_unicos
0,738000377,Unicentro Cali,354
1,737998204,Versalles,328
2,737999419,El Sena,287
3,737998095,Unidad Residencial Santiago de Cali,276
4,737998838,Calima,250


In [7]:
conn.close()